# WHOBS-IE-100
## Notionally Optimal, 100% Wind+Hydrogen+Other+Battery+Solar Electricity System for Ireland (IE)

- **Project:** [OESM-IE](http://ecrn.eeng.dcu.ie/projects/oesm-ie)
- **Funding:** [Sustainable Energy Authority of Ireland (SEAI) Research, Development and Demonstration Programme](https://www.seai.ie/grants/research-funding/research-development-and-demonstration-fund/), award reference SEAI RDD/00246 2018.
- **Author:** Barry McMullin, barry.mcmullin@dcu.ie
- **Last modified:** 27 May 2020
- **© 2020:** [Dublin City University](http://www.dcu.ie/)
- **Licence:** [GNU GENERAL PUBLIC LICENSE Version 3](https://www.gnu.org/licenses/gpl-3.0.en.html)

This is a derived from: [Optimal Wind+Hydrogen+Other+Battery+Solar (WHOBS) electricity systems for European countries](https://github.com/PyPSA/WHOBS)

Download the [Jupyter notebook](https://jupyter.org/) at: **TODO**


## Introduction/motivation

[WHOBS](https://github.com/PyPSA/WHOBS), and the associated interactive web application [model.energy](https://model.energy/), allows modelling of notionally optimised "firm" electricity generation for a given level of (constant/"baseload") capacity, based exclusively on *variable* renewable (VRE: here limited to wind and solar) sources, coupled with hydrogen and/or battery storage (to cover "when the wind doesn't blow and the sun doesn't shine"). 

Wind and solar resource variability is configured for states within Europe using historical data from [Renewables.ninja](https://www.renewables.ninja/) ([model.energy](https://model.energy/) extends this coverage to global geographical locations using other data sources).

**WHOBS-IE-100** adapts WHOBS to model delivery of **100%** of electricity demand for one particular European country (**Ireland**), based on [historical load data](http://www.eirgridgroup.com/how-the-grid-works/renewables/) from the [Irish Transmission System Operator (TSO) eirgrid](http://www.eirgridgroup.com/).

This result illustrates the (rough) trade-off between:

- Raw VRE *overprovision* (building more VRE capacity than can be directly dispatched at all times, but meaning that more demand can be covered directly by instantaneous VRE generations)
- Dispatch down (discarding some generation when it is in excess of instantaneous load)
- Storage (storing some generation when it is in excess of instantaneous load)
    - Short term, high efficiency storage: battery
    - Long term, low efficiency storage: hydrogen

This toy model assumes a completely isolated grid (no external interconnection).


## TODO

+ Create/release as github project: add link in metadata block at top?
+ Link to the [detailed caveats in the WHOBS README](https://github.com/PyPSA/WHOBS#warnings).
+ Add more cautions/, including:
    - *multi-annual* variability in generation and load
    - constraint as well as curtailment 
    - system stability (traditional SNSP!) 
    - onshore wind only (offshore would be less variable but generally still more expensive)
    - limited/no salt cavern storage within the IE jurisdiction (there is some in NI, but [currently targetted for NG storage!](https://www.infrastrataplc.com/projects/islandmagee-energy/)
    - unclear whether the proposed (solved) wind and solar capacity is even feasible within the jurisdiction
    - certain "low carbon", "firm" generation sources are deliberately omitted: nuclear, fossil fuel with CCS and bioenergy: it is likely that a (near) zero-$CO_2$ system could be achieved at lower cost by allowing such a wider range of options
    - etc. etc...
+ Batteries are allowed a tech-generic lifetime of 25 years (in assumptions .csv) - which seems a bit generous? Though may not change the results all that much...
+ Add some (simplistic) model of interconnection: e.g. as a `StorageUnit` with `cyclic=True`, fixed `p_nom`, efficiency somewhat reflective of real IC losses (to UK/France). This is easier than trying to pick a price for a market import/export model (though it excludes nett exports as a trade opportunity...). Skates over the NI integration connection, which arguably deserves finer grained representation (given similar wind var profile).
+ Add some representation of offshore wind as well as onshore; set `p_nom_extendable_max` for onshore, based on actual SEAI estimates. Note that offshore variability pattern is expected to be different from onshore (potential high capacity factor, absent dispatch down).
+ Add some notional representation of grid losses (matching typical SEAI/Eirgrid levels?).
+ Add some crude capability of changing average load level for target year (linear, exponential, whatever...)
+ Add more flexible options on temporal resolution - not just 1 or 3 hours, but 6, 12, 24 hours?
+ Add H2P OCGT (as well as CCGT)? Let `lopf()` optimise between OCGT and CCGT (will it all go to one?).
+ Instead of a H2 steel tank storage being binary, leave it available unconditionally, but (optionally) set an `e_extendable_max` limit on H2 salt cavern storage (possibly loosely based on actual geology available in NI).
+ Add a "middle tier" H2 storage option (between steel tank and salt cavern): [loosely based on ammonia tank storage](https://ammoniaindustry.com/ammonia-for-energy-storage-economic-and-technical-analysis/)?
+ Of course, extending into heating and transport sectors is also on the agenda!
+ Refactor the "scenario" concept:
    - Incorporate `frequency` (properly, `time_resolution` or `time_step`?).
    - Incorporate dispatch priority among VRE (solar/wind) as a scenario variable
    - Currently "hydrogen" and "hydrogen steel tank" are effectively boolean scenario variables, though not expressly incorporated in the scenario mechanism/naming. Maybe make "all" the discrete technologies boolean in this sense, with a scenario tech substring something like: `WSHTB` (Wind|Solar|H2|Tank|Battery)?
    - Refactor so that generic `network` only constructed once, and then specific details varied by scenario; should yield a little performance improvement/responsivity...
+ Possibly refactor to remove the `ct` ("country"?) use? This is inherited from WHOBS, which genuinely has configuration capability for a variety of countries; but WHOBS-IE-100 really is hard-wired already for IE?
+ Figure out/refactor exactly the way the plotting works...
+ Refactor the import of third-party data files (Renewables.ninja and Eirgrid) to automate, with local caching...
+ Create a single line diagram illustration (SLD)
+ Screencast tutorial/explanation?


In [1]:
import pypsa
import pandas as pd
idx = pd.IndexSlice

import matplotlib.pyplot as plt
from pyomo.environ import Constraint

%matplotlib inline

In [2]:
def fmt_float(x) :
    return (F"{x: 6.2f}")
    
pd.set_option('float_format', fmt_float)

In [3]:
# Initialise empty DataFrame to collect run configuration
run_configs = pd.DataFrame()

# Initialise empty DataFrame to collect run output stats
run_stats = pd.DataFrame()

In [4]:
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
plt.rcParams["figure.figsize"] = (12,6) 

## Major configuration settings

In [37]:
# WHOBS was generic to multiple countries coded by `ct`; but here we will be 
# hard-wired to `IE` in other ways, so set `ct` to match...
ct = "IE"

# year(s) for weather data (solar is 1985-2015 inclusive, wind is 1980-2016)
weather_year_start = 2010
weather_year_end = 2010

# year(s) for load data
load_year_start = 2015
load_year_end = 2015

#1 is hourly, 3 is 3-hourly
frequency = 12

#change to cbc or glpk for open-source solvers
solver_name = "cbc"

## Required data

### Wind and solar resource variabilities

From [Renewables.ninja Downloads](https://www.renewables.ninja/downloads):

- Solar time series "ninja_pv_europe_v1.1_sarah.csv" from [PV v1.1 Europe (.zip)](https://www.renewables.ninja/static/downloads/ninja_europe_pv_v1.1.zip)
- Wind time series "ninja_wind_europe_v1.1_current_on-offshore.csv" from [Wind v1.1 Europe (.zip)](https://www.renewables.ninja/static/downloads/ninja_europe_wind_v1.1.zip)

### IE Load (electricity demand) variability

From [eirgrid System and Renewable Data Reports](http://www.eirgridgroup.com/how-the-grid-works/renewables/):

- [System-Data-Qtr-Hourly-2018-2019.xlsx](http://www.eirgridgroup.com/site-files/library/EirGrid/System-Data-Qtr-Hourly-2018-2019.xlsx) 
- [System-Data-Qtr-Hourly-2016-2017.xlsx](http://www.eirgridgroup.com/site-files/library/EirGrid/System-Data-Qtr-Hourly-2016-2017.xlsx)
- [System-Data-Qtr-Hourly-2014-2015.xlsx](http://www.eirgridgroup.com/site-files/library/EirGrid/System-Data-Qtr-Hourly-2014-2015.xlsx)


## Read in wind and solar variability data

**TODO:** Ideally, recode this to check for local copy, and, if not available, automatically download 
and extract the required .csv from the .zip in each case; but for the moment, just assume there is are local copies of the .csv files already available.

**TODO:** An alterative to using renewables ninja (specfically for wind) would be to extract the variability data (of actual wind generation) from historical eirgrid data. This would reflect the performance of the IE wind fleet as of whatever historical date was used: which may be a good thing or a bad thing of course (since that is almost 100% onshore for the moment, it is "biased against" offshore - arguably?).


In [6]:
#rninja_base_url = "https://www.renewables.ninja/static/downloads/"
r_ninja_base_url = '../renewables-ninja/' # Actually already downloaded...

In [7]:
#solar_pv_zip_file = 'ninja_europe_pv_v1.1.zip'
#solar_pv_zip_url = r_ninja_base_url + solar_pv_zip_file

solar_pv_csv_file = 'ninja_pv_europe_v1.1_sarah.csv'
solar_pv_csv_url = r_ninja_base_url + solar_pv_csv_file

#read in renewables.ninja solar time series
solar_pu_raw = pd.read_csv(solar_pv_csv_url,
                       index_col=0,parse_dates=True)
solar_pu = solar_pu_raw.resample(str(frequency)+"H").mean()

In [8]:
#wind_zip_file = 'ninja_europe_wind_v1.1.zip'
#wind_zip_url = r_ninja_base_url + wind_zip_file

wind_csv_file = 'ninja_wind_europe_v1.1_current_on-offshore.csv'
wind_csv_url = r_ninja_base_url + wind_csv_file

#read in renewables.ninja wind time series
wind_pu_raw = pd.read_csv(wind_csv_url,
                       index_col=0,parse_dates=True)
wind_pu = wind_pu_raw.resample(str(frequency)+"H").mean()

## Read in and preprocess load variability data (via Ireland TSO, [EirGrid](http://www.eirgridgroup.com/))

We start with [historical data inputs from EirGrid](http://www.eirgridgroup.com/how-the-grid-works/renewables/) which show 15-minute time series for:

- wind availability
- wind generation
- total generation
- total load

broken out by:

- IE (Republic of Ireland) only
- NI (Northern Ireland) only

As this particular notebook in only relying on this data for an example of "typical" annual variability in aggregate load we select the data for **IE (Republic of Ireland) only**.

The data will also be downsampled from 15-minute to hourly-resolution (via simple averaging of each hour).

In [9]:
# Retrieve example eirgrid system data into a pd.dataframe

# If file already available locally, can point at that; otherwise use the web url
# (i.e. uncomment one or the other of the following two statements).

#eirgrid_base_url = "http://www.eirgridgroup.com/site-files/library/EirGrid/"
eirgrid_base_url = "../eirgrid/"

load_data_filename = "System-Data-Qtr-Hourly-2014-2015.xlsx"
load_data_url = eirgrid_base_url + load_data_filename

cols = ['DateTime', 'GMT Offset', "IE Demand"]

load_data_raw = pd.read_excel(load_data_url, usecols = cols)

In [10]:
#print(load_data_raw.dtypes)
print(load_data_raw)

                 DateTime  GMT Offset  IE Demand
0     2014-01-01 00:00:00           0    2898.72
1     2014-01-01 00:15:00           0    2868.97
2     2014-01-01 00:30:00           0    2826.42
3     2014-01-01 00:45:00           0    2786.94
4     2014-01-01 01:00:00           0    2723.94
...                   ...         ...        ...
70075 2015-12-31 22:45:00           0    2959.23
70076 2015-12-31 23:00:00           0    2993.32
70077 2015-12-31 23:15:00           0    3078.19
70078 2015-12-31 23:30:00           0    3055.56
70079 2015-12-31 23:45:00           0    3018.76

[70080 rows x 3 columns]


## Fix the timestamps...

The raw eirgrid data has one column showing localtime (`DateTime`, type `pd.Timestamp`, holding "naive" timestamps - no recorded timezone) and a separate column showing the offset, in hours, from UTC for each individual row (`GMT Offset`). It will be simpler here to convert all the `DateTime` values to UTC (and explicitly having the UTC timezone).

We can then dispense with the `GMT Offset` column as it is redundant.

In [11]:
from datetime import timedelta

def tz_fix(row):
  try:
    naive_timestamp = row['DateTime']
    gmt_offset = row['GMT Offset'] 
    utc_timestamp = naive_timestamp - timedelta(hours=float(gmt_offset))
        # float() conversion required for timedelta() argument!
        # Must SUBTRACT the GMT Offset to get GMT/UTC
    row['DateTime'] = utc_timestamp.tz_localize('UTC')
  except Exception as inst:
    print(F"Exception:\n {row}")
    print(inst)
  return row

# This may be rather be slow for a big dataset...
load_data_raw = load_data_raw.apply(tz_fix, axis=1).drop(columns='GMT Offset')

load_data_raw.set_index('DateTime', verify_integrity=True, inplace=True)

## Extract period of interest

In [43]:
load_date_start = "{}-01-01 00:00:00+0".format(load_year_start)
load_date_end = "{}-12-31 23:45:00+0".format(load_year_end)

load_data_raw = load_data_raw.loc[load_date_start:load_date_end]

print(load_data_raw)

load_raw = load_data_raw.loc[:,'IE Demand'] # convert to pd.Series

load_raw_max = load_raw.max()
load_raw_mean = load_raw.mean()
load_raw_min = load_raw.min()
load_raw_e = load_raw.sum()*0.25 # Raw time interval is 15m == 0.25h

print(F"load_raw_max: {(load_raw_max/1.0e3) : 6.3f} GW")
print(F"load_raw_mean: {(load_raw_mean/1.0e3) : 6.3f} GW")
print(F"load_raw_min: {(load_raw_min/1.0e3) : 6.3f} GW")
print(F"load_raw_e: {(load_raw_e/1.0e6) : 6.3f} TWh")

                           IE Demand
DateTime                            
2015-01-01 00:00:00+00:00    2903.34
2015-01-01 00:15:00+00:00    2885.45
2015-01-01 00:30:00+00:00    2832.37
2015-01-01 00:45:00+00:00    2775.59
2015-01-01 01:00:00+00:00    2726.20
...                              ...
2015-12-31 22:45:00+00:00    2959.23
2015-12-31 23:00:00+00:00    2993.32
2015-12-31 23:15:00+00:00    3078.19
2015-12-31 23:30:00+00:00    3055.56
2015-12-31 23:45:00+00:00    3018.76

[35040 rows x 1 columns]
load_raw_max:  4.704 GW
load_raw_mean:  3.034 GW
load_raw_min:  1.759 GW
load_raw_e:  26.576 TWh


## Data quality checks?

Minimal data quality check: make sure [we have no missing values](https://chartio.com/resources/tutorials/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe/) (either `None` or `NaN`).

In [14]:
assert(not load_data_raw.isnull().values.any())

## Configure the load timeseries

This is derived from the IE empirical/historical `Demand`. 

In [15]:
load_data = load_data_raw.resample(str(frequency)+"H").mean()

load = load_data.loc[:,'IE Demand'] # convert to pd.Series

load_max = load.max()
load_mean = load.mean()
load_min = load.min()
load_e = load.sum()*frequency

In [16]:
print(load)

print(F"load_max: {(load_max/1.0e3) : 6.3f} GW")
print(F"load_mean: {(load_mean/1.0e3) : 6.3f} GW")
print(F"load_min: {(load_min/1.0e3) : 6.3f} GW")
print(F"load_e: {(load_e/1.0e6) : 6.3f} TWh")

DateTime
2015-01-01 00:00:00+00:00    2414.56
2015-01-01 12:00:00+00:00    3191.29
2015-01-02 00:00:00+00:00    2562.62
2015-01-02 12:00:00+00:00    3495.36
2015-01-03 00:00:00+00:00    2551.41
                              ...   
2015-12-29 12:00:00+00:00    3570.23
2015-12-30 00:00:00+00:00    2676.53
2015-12-30 12:00:00+00:00    3513.35
2015-12-31 00:00:00+00:00    2634.46
2015-12-31 12:00:00+00:00    3477.54
Freq: 12H, Name: IE Demand, Length: 730, dtype: float64
load_max:  4.073 GW
load_mean:  3.034 GW
load_min:  2.215 GW
load_e:  26.576 TWh


## Required functions

In [17]:
# TODO: need some docs/explanation/sources for this calculation?

def annuity(lifetime, rate):
    if rate == 0.0 :
        return 1.0/lifetime
    else:
        return rate/(1.0 - (1.0 / (1.0 + rate)**lifetime))

In [18]:
def prepare_assumptions(Nyears=1,usd_to_eur=1/1.2,assumptions_year=2020):
    """set all asset assumptions and other parameters"""

    assumptions = pd.read_csv("assumptions.csv",index_col=list(range(3))).sort_index()

    #correct units to MW and EUR
    assumptions.loc[assumptions.unit.str.contains("/kW"),"value"]*=1e3
    assumptions.loc[assumptions.unit.str.contains("USD"),"value"]*=usd_to_eur

    assumptions = assumptions.loc[idx[:,assumptions_year,:],
                                  "value"].unstack(level=2).groupby(level="technology").sum(min_count=1)

    #fill defaults
    assumptions = assumptions.fillna({"FOM" : assumptions.at["default","FOM"],
                                      "discount rate" : assumptions.at["default","discount rate"],
                                      "lifetime" : assumptions.at["default","lifetime"]})

    #annualise investment costs, add FOM
    assumptions["fixed"] = [(annuity(v["lifetime"],v["discount rate"]) + 
                             v["FOM"]/100.)*v["investment"]*Nyears for i,v in assumptions.iterrows()]

    return assumptions

In [35]:
def solve_network(run_config):

    frequency = run_config['frequency']
    weather_year_start = run_config['weather_year_start']
    weather_year_end = run_config['weather_year_end']
    Nyears = weather_year_end - weather_year_start + 1

#    assumptions_year = int(scenario[:4])

    assumptions = prepare_assumptions(Nyears=Nyears,
                                      assumptions_year=run_config['assumptions_year'])

#    if "steel_tanks" in scenario:
    if run_config['steel_tanks'] :
        assumptions.loc["H2 storage"] = assumptions.loc["H2 steel tank storage"]
    else:
        assumptions.loc["H2 storage"] = assumptions.loc["H2 underground storage"]


    network = pypsa.Network()

    snapshots = pd.date_range("{}-01-01".format(weather_year_start),
                              "{}-12-31 23:00".format(weather_year_end),
                              freq=str(frequency)+"H")

    network.set_snapshots(snapshots)

    network.snapshot_weightings = pd.Series(float(frequency),index=network.snapshots)

    network.add("Bus",ct)
    network.add("Load",ct,
                bus=ct,
                p_set= load.values)

    network.add("Generator",ct+" solar",
                bus=ct,
                p_max_pu = solar_pu[ct],
                p_nom_extendable = True,
                marginal_cost = 0.02, 
                #Small cost to prefer curtailment to destroying energy in storage, wind curtails before solar
                capital_cost = assumptions.at['utility solar PV','fixed'],
                #p_nom_max = 0.0
               )

    network.add("Generator",ct+" wind",
                bus=ct,
                p_max_pu = wind_pu[ct+"_ON"],
                p_nom_extendable = True,
                marginal_cost = 0.01, 
                #Small cost to prefer curtailment to destroying energy in storage, wind curtails before solar
                capital_cost = assumptions.at['onshore wind','fixed'])

    network.add("Bus",ct + " battery")

    network.add("Store",ct + " battery storage",
                bus = ct + " battery",
                e_nom_extendable = True,
                e_cyclic=True,
                capital_cost=assumptions.at['battery storage','fixed'])

    network.add("Link",ct + " battery charge",
                bus0 = ct,
                bus1 = ct + " battery",
                efficiency = assumptions.at['battery inverter','efficiency'],
                p_nom_extendable = True,
                capital_cost=assumptions.at['battery inverter','fixed'])

    network.add("Link",ct + " battery discharge",
                bus0 = ct + " battery",
                bus1 = ct,
                p_nom_extendable = True,
                efficiency = assumptions.at['battery inverter','efficiency'])

    def extra_functionality(network,snapshots):
        def battery(model):
            return (model.link_p_nom[ct + " battery charge"] 
                    == model.link_p_nom[ct + " battery discharge"]*network.links.at[ct 
                    + " battery charge","efficiency"])

        network.model.battery = Constraint(rule=battery)

    network.add("Bus",
                     ct + " H2",
                     carrier="H2")

    network.add("Link",
                    ct + " H2 electrolysis",
                    bus1=ct + " H2",
                    bus0=ct,
                    p_nom_extendable=True,
                    efficiency=assumptions.at["H2 electrolysis","efficiency"],
                    capital_cost=assumptions.at["H2 electrolysis","fixed"])

    network.add("Link",
                     ct + " H2 to power",
                     bus0=ct + " H2",
                     bus1=ct,
                     p_nom_extendable=True,
                     efficiency=assumptions.at["H2 CCGT","efficiency"],
                     capital_cost=assumptions.at["H2 CCGT","fixed"]*assumptions.at["H2 CCGT","efficiency"])  
                     #NB: fixed cost is per MWel

    network.add("Store",
                     ct + " H2 storage",
                     bus=ct + " H2",
                     e_nom_extendable=True,
                     e_cyclic=True,
                     capital_cost=assumptions.at["H2 storage","fixed"])


    if solver_name == "gurobi":
        solver_options = {"threads" : 4,
                          "method" : 2,
                          "crossover" : 0,
                          "BarConvTol": 1.e-5,
                          "FeasibilityTol": 1.e-6 }
    else:
        solver_options = {}


    network.consistency_check()

#    network.lopf(solver_name=solver_name,
#                 solver_options=solver_options,
#                 extra_functionality=extra_functionality)

    return network

In [20]:
def gather_run_stats(run_stats,network,ct,full_scenario):
    
    total_load_e = (network.loads_t.p.sum().sum() * frequency)
    available_e = network.generators_t.p_max_pu.multiply(network.generators.p_nom_opt).sum() * frequency
    total_available_e = available_e.sum()
    dispatched_e = network.generators_t.p.sum() * frequency
    total_dispatched_e = dispatched_e.sum()
    undispatched_e = (available_e - dispatched_e)
    total_undispatched_e = undispatched_e.sum()
    undispatched_frac = undispatched_e/available_e
    
    run_stats.loc["System load (TWh)",full_scenario] = total_load_e/1.0e6
    run_stats.loc["System available (TWh)",full_scenario] = total_available_e/1.0e6
    run_stats.loc["System efficiency gross (%)",full_scenario] = (total_load_e/total_available_e)*100.0
        # "gross" includes dispatch down
    run_stats.loc["System dispatched (TWh)",full_scenario] = total_dispatched_e/1.0e6
    run_stats.loc["System dispatched down (TWh)",full_scenario] = total_undispatched_e/1.0e6
    run_stats.loc["System storage loss (TWh)",full_scenario] = (total_dispatched_e-total_load_e)/1.0e6

    run_stats.loc["System efficiency net (%)",full_scenario] = (total_load_e/total_dispatched_e)*100.0
        # "net" of dispatch down
    run_stats.loc["System LCOE (€/MWh)",full_scenario] = network.buses_t.marginal_price.mean()[ct]
    
    total_hours = network.snapshot_weightings.sum()
    
    gens = ["wind","solar"]
    for g in gens:
        g_idx = ct + " " + g
        run_stats.loc[g+" capacity nom (GW)",full_scenario] = (
            network.generators.p_nom_opt[g_idx]/1.0e3)
        run_stats.loc[g+" available (TWh)",full_scenario] = available_e[g_idx]/1.0e6
        run_stats.loc[g+" dispatched (TWh)",full_scenario] = dispatched_e[g_idx]/1.0e6
        run_stats.loc[g+" penetration (%)",full_scenario] = (dispatched_e[g_idx]/total_dispatched_e)*100.0 
        run_stats.loc[g+" dispatched down (TWh)",full_scenario] = (undispatched_e[g_idx])/1.0e6
        run_stats.loc[g+" dispatched down (%)",full_scenario] = (undispatched_frac[g_idx])*100.0
        run_stats.loc[g+" capacity factor max (%)",full_scenario] = (
            network.generators_t.p_max_pu[g_idx].mean())*100.0
        run_stats.loc[g+" capacity factor act (%)",full_scenario] = (
            dispatched_e[g_idx]/(network.generators.p_nom_opt[g_idx]*total_hours))*100.0
        
    run_stats.loc["Battery charge/discharge (MW)",full_scenario] = network.links.p_nom_opt[ct + " battery charge"]
        # NB: battery charge are discharge p_nom are constrained to be equal, modulo efficiency
    run_stats.loc["Battery storage (MWh)",full_scenario] = network.stores.e_nom_opt[ct + " battery storage"]
    run_stats.loc["P2H (electrolysis, GW)",full_scenario] = (network.links.p_nom_opt[ct + " H2 electrolysis"]/1.0e3)
    run_stats.loc["H2P (H2 CCGT, GW)",full_scenario] = (network.links.p_nom_opt[ct + " H2 to power"]/1.0e3)
    run_stats.loc["H2 storage (TWh)",full_scenario] = (network.stores.e_nom_opt[ct + " H2 storage"]/1.0e6)

    return run_stats

## Solve the system

In [36]:
#

#scenario = "2030-steel_tanks"
#scenario = "2030-salt_cavern"

# Hardwired: IE, 2030 costs, 2010 wind and solar, 2015 load, wind dispatch priority, 
#full_scenario = ct+'-'+scenario+'-WP2010'+'-'+str(frequency)+"H"

#network = solve_network(ct, scenario)

run_config = pd.Series({
    'run_name' : 'Testing!',
    'frequency' : frequency,
    'weather_year_start' : 2010,
    'weather_year_end' : 2010,
    'load_year_start' : load_year_start,
    'load_year_end' : load_year_end,
    'assumptions_year' : 2030,
    'steel_tanks' : False,
    'h_store_max_e' : 1.0e12 #MWh
    })

display(run_config)

network = solve_network(run_config)

run_name                    Testing!
assumptions_year                2030
steel_tanks                    False
h_store_max_e       1000000000000.00
dtype: object

In [22]:
# Uncomment and run this if/when desired to clear the accumulated stats.

# FIXME: would like to have any way to shovel these result checkpoints out to 
# an external (persistent) data file?

# stats_checkpoint_0_pre_VRE_freq_mean = stats # manually fix/choose archive name!
#run_stats=pd.DataFrame() 

In [23]:
gather_run_stats(run_stats,network,ct,'Run 0')
run_stats

,Run 0
System load (TWh),26.58
System available (TWh),46.27
System efficiency gross (%),57.44
System dispatched (TWh),29.47
System dispatched down (TWh),16.80
System storage loss (TWh),2.89
System efficiency net (%),90.19
System LCOE (€/MWh),118.81
wind capacity nom (GW),19.98
wind available (TWh),39.17


## Extra outputs (WHOBS legacy)

**FIXME:** Need to review which - if any - of these are still relevant in WHOBS-IE-100

In [ ]:
#absolute market value in EUR/MWh
(network.generators_t.p.multiply(network.buses_t.marginal_price[ct],axis=0).sum()/network.generators_t.p.sum())

In [ ]:
# market cost in EUR/MWh (LCOE) - 
# for long-term equilibrium without additional constraints, same as market value
(network.generators.capital_cost*network.generators.p_nom_opt)/network.generators_t.p.sum()

In [ ]:
#relative market value
(network.generators_t.p.multiply(network.buses_t.marginal_price[ct],axis=0).sum()/
     network.generators_t.p.sum())/network.buses_t.marginal_price[ct].mean()

In [ ]:
#relative market value
(network.links_t.p0.multiply(network.buses_t.marginal_price[ct],axis=0).sum()/
     network.links_t.p0.sum())/network.buses_t.marginal_price[ct].mean()

In [ ]:
%matplotlib notebook

#start = "2015-01-01"
#stop = "2015-12-31"
start = F"{year_start}-01-01"
stop = F"{year_end}-12-31"

fig, ax = plt.subplots()
#fig.set_size_inches((10,8))


rename = {"wind" : "onshore wind",
          "solar" : "utility solar PV",
          "battery discharge" : "battery discharge",
          "battery charge" : "battery charge",
          "H2 electrolysis" : "hydrogen electrolysis",
          "H2 to power" : "hydrogen turbine"}

rename = {ct + " "+k : v for k,v in rename.items()}

rename[ct] = "load"

colors = {"onshore wind" : "b",
           "utility solar PV" : "y",
           "battery discharge" : "gray",
           "battery charge" : "gray",
           "load" : "k",
           "hydrogen electrolysis" : "m",
           "hydrogen turbine" : "r"
          }


# FIXME: for some reason (solver tolerance?) some solution values that should be strictly 
# positive or negative may be infinitesimally (< 10e-10) of the other sign. This will cause 
# df.plot(stacked=True) to throw an error (requires all values to have same sign, positive or 
# negative). So we do an ugly ".round(10)" to fix it...

positive = pd.concat((network.generators_t.p,-network.links_t.p1[[ct+" H2 to power",ct+" battery discharge"]]),
                     axis=1).rename(columns=rename).round(10)
negative = pd.concat((-network.loads_t.p,-network.links_t.p0[[ct+" H2 electrolysis",ct+" battery charge"]]),
                     axis=1).rename(columns=rename).round(10)

print((abs(positive.sum(axis=1) + negative.sum(axis=1)) > 0.1).any())

positive.loc[start:stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                              color=[colors[i] for i in positive.columns])

negative.loc[start:stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                              color=[colors[i] for i in negative.columns])

ax.set_ylim([-2.0*load_max,+2.0*load_max])
ax.set_xlim([start,stop])
ax.set_ylabel("Dispatch (generation is +ve, demand is -ve) [MW]")
ax.legend(ncol=3,loc="upper left")

fig.tight_layout()

#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

In [ ]:
network.buses_t.marginal_price.plot()

In [ ]:
network.buses_t.marginal_price.mean()

In [ ]:
network.links_t.p0.mean()/network.links.p_nom_opt

In [ ]:
%matplotlib notebook

opt_costs = pd.Series()


opt_costs = pd.concat((opt_costs,network.generators.capital_cost*network.generators.p_nom_opt))

opt_costs = pd.concat((opt_costs,network.links.capital_cost*network.links.p_nom_opt))

opt_costs = pd.concat((opt_costs,network.stores.capital_cost*network.stores.e_nom_opt))


(opt_costs/opt_costs.sum()).plot(kind="bar",grid=True)


In [ ]:
print(opt_costs)

## Reflections (random, possibly defunct!)

+ I don't really have a good intuition for why battery storage is being used at the level it is. Presumably if we go to more coarse-grained temporal resolution it stops being used altogether?
+ Battery and DSM (including heating and vehicle charging flexibilities) presumablly all fall into the same ~24 hour flexibility regime.
+ Flow batteries would, presumably, have a quite different profile: maybe conceivably competitive with ammonia? Could we add a model of that?